<h1>Proyecto sobre interpretabilidad sobre modelos de caja negra</h1>

<p>En este proyecto nos dedicaremos a realizar el entrenamiento para los modelos de interpretabilidad de modelos de caja negra,en nuestro proyecto hemos realizado hacer el entrenamiento de modelos para tareas de clasifiacacion. Adelante en esta practica primero realizaremos el entrenamiento de los diferentes modelos de caja negra primero realizaremos el entrenamiento a un modelo de redes neuronales.</p>
<u><b>A continuacion el siguiente bloque de codigo es necesario ejecutarlo para el correcto funcionamiento de este notebook.</b></u>

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
from tensorflow import get_logger
get_logger().setLevel('ERROR')
import random
from tensorflow import random as tensorflow_random

random.seed(2398572)
tensorflow_random.set_seed(394867)

import pandas
from sklearn import model_selection

np.random.seed(43958734)
np.set_printoptions(threshold=10)
from tensorflow import keras

from scipy.spatial.distance import euclidean
from scipy.spatial.distance import cdist

<p>Ahora vamos a seleccionar el primer csv que contiene los datos y seleccionaremos los atributos que tendremos en cuenta para hacer el entrenamiento del algoritmo</p>

In [4]:
shill_bidding = pandas.read_csv("Shill Bidding Dataset.csv")
shill_bidding.head()

,Record_ID,Auction_ID,Bidder_ID,Bidder_Tendency,Bidding_Ratio,Successive_Outbidding,Last_Bidding,Auction_Bids,Starting_Price_Average,Early_Bidding,Winning_Ratio,Auction_Duration,Class
0,1,732,_***i,0.200000,0.400000,0.0,0.000028,0.0,0.993593,0.000028,0.666667,5,0
1,2,732,g***r,0.024390,0.200000,0.0,0.013123,0.0,0.993593,0.013123,0.944444,5,0
2,3,732,t***p,0.142857,0.200000,0.0,0.003042,0.0,0.993593,0.003042,1.000000,5,0
3,4,732,7***n,0.100000,0.200000,0.0,0.097477,0.0,0.993593,0.097477,1.000000,5,0
4,5,900,z***z,0.051282,0.222222,0.0,0.001318,0.0,0.000000,0.001242,0.500000,7,0


Seleccionamos los atributos a tener encuenta a la hora de entrenar

In [5]:
atributos = shill_bidding.loc[:,'Bidder_Tendency':'Auction_Duration']
atributos = atributos.to_numpy()
print(atributos)


[[2.00000000e-01 4.00000000e-01 0.00000000e+00 ... 2.77778000e-05
  6.66666667e-01 5.00000000e+00]
 [2.43902440e-02 2.00000000e-01 0.00000000e+00 ... 1.31226852e-02
  9.44444444e-01 5.00000000e+00]
 [1.42857143e-01 2.00000000e-01 0.00000000e+00 ... 3.04166670e-03
  1.00000000e+00 5.00000000e+00]
 ...
 [5.55555560e-02 4.34782610e-02 0.00000000e+00 ... 1.56630291e-02
  0.00000000e+00 7.00000000e+00]
 [7.69230770e-02 8.69565220e-02 0.00000000e+00 ... 4.15013200e-04
  0.00000000e+00 7.00000000e+00]
 [1.63934430e-02 4.34782610e-02 0.00000000e+00 ... 3.40350529e-01
  0.00000000e+00 7.00000000e+00]]


<p>Vamos a seleccionar el atributo objetivo para el algoritmo</p>

In [6]:
objetivo = shill_bidding['Class']
objetivo = objetivo.to_numpy()
print(objetivo)

[0 0 0 ... 0 0 0]


Entrenamos el modelo de redes neuronales y dividimos el modelo de entrenamiento con el modelo de pruebas

In [7]:
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = model_selection.train_test_split(
    atributos, objetivo, test_size=.15)

Input indica el numero de dimensiones y dense indica cantidad de neuronas y la funcion de activacion, que por defecto es la identidad. Este ejemplo es para modelos de <b>clasificacion</b> 

La red neuronal posee una única capa (aparte de la de entrada) que proporciona como salida un array bidimensional None x 1 (es decir, para cada ejemplo proporciona un valor) y 9 parámetros (el peso de la conexión de cada entrada con la neurona de la capa más el sesgo de la neurona) entrenables (la red los aprenderá mediante el algoritmo de entrenamiento).

In [8]:
red_bidding = keras.Sequential()
red_bidding.add(keras.Input(shape=(9,)))
red_bidding.add(keras.layers.Dense(10, activation='relu'))
red_bidding.add(keras.layers.Dense(1, activation='sigmoid'))

Los pesos y sesgos de la red se guardan en el atributo weights (son los arrays asociados al argumento numpy en la estructura de datos guardada en ese atributo).

In [9]:
red_bidding.summary()
red_bidding.weights

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                100       
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________


[<tf.Variable 'dense/kernel:0' shape=(9, 10) dtype=float32, numpy=
 array([[ 0.16066974, -0.1832436 , -0.3538186 , ...,  0.340855  ,
          0.25578886,  0.5128347 ],
        [-0.1894975 , -0.0651882 , -0.12789819, ..., -0.19957277,
         -0.0312627 , -0.17042843],
        [-0.4715354 , -0.20407248,  0.29305267, ..., -0.50047326,
          0.43153203,  0.34140068],
        ...,
        [-0.01709646, -0.03771824, -0.15322047, ...,  0.49320394,
          0.24029279, -0.05415136],
        [ 0.2125234 ,  0.41607064, -0.06427649, ..., -0.03796834,
          0.12062675,  0.19724232],
        [ 0.27244765,  0.48618132,  0.47343248, ...,  0.20641726,
          0.0910573 ,  0.37640864]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'dense_1/kernel:0' shape=(10, 1) dtype=float32, numpy=
 array([[ 0.57531995],
        [ 0.17448515],
        [-0.48876932],
        [ 0.38587934],
  

Entrenamos la red neuronal con un optimizador y una funcion de perdida, en este caso al tratarse de un modelo de clasifiacion seleccionamos la funcion de perdida como la entropia cruzada binaria

In [10]:
red_bidding.compile(optimizer='SGD', loss='binary_crossentropy',
                   metrics=['accuracy'])

Aqui entrenamos el modelo con los atributos de prueba seleccionados y el objetivo de entrenamiento

In [11]:
red_bidding.fit(atributos_entrenamiento, objetivo_entrenamiento,
               batch_size=10, epochs=10)

Epoch 1/10
538/538 [==============================] - 1s 1ms/step - loss: 0.3997 - accuracy: 0.8773
Epoch 2/10
538/538 [==============================] - 1s 1ms/step - loss: 0.3068 - accuracy: 0.8954
Epoch 3/10
538/538 [==============================] - 1s 1ms/step - loss: 0.2324 - accuracy: 0.9149
Epoch 4/10
538/538 [==============================] - 1s 1ms/step - loss: 0.1664 - accuracy: 0.9509
Epoch 5/10
538/538 [==============================] - 1s 1ms/step - loss: 0.1228 - accuracy: 0.9630
Epoch 6/10
538/538 [==============================] - 1s 1ms/step - loss: 0.0970 - accuracy: 0.9635
Epoch 7/10
538/538 [==============================] - 1s 1ms/step - loss: 0.0829 - accuracy: 0.9631
Epoch 8/10
538/538 [==============================] - 1s 1ms/step - loss: 0.0740 - accuracy: 0.9671
Epoch 9/10
538/538 [==============================] - 1s 1ms/step - loss: 0.0686 - accuracy: 0.9678
Epoch 10/10
538/538 [==============================] - 1s 1ms/step - loss: 0.0644 - accuracy: 0.9685

Y ahora vamos a evaluar el modelo y ejecutar su prediccion con los atributos de prueba

In [12]:
red_bidding.evaluate(atributos_prueba, objetivo_prueba)
red_bidding.predict(atributos_prueba)
j = 0
vp=0
vn=0
fp=0
fn=0
for i in red_bidding.predict(atributos_prueba):
    
    if(i > 0.5):
        i = 1
    elif(i <= 0.5):
        i = 0
    if(i == objetivo_prueba[j] and i == 1):
        vp+=1
    elif(i !=objetivo_prueba[j] and i == 1):
        fp +=1
    elif(i ==objetivo_prueba[j] and i == 0):
        vn +=1
    elif(i !=objetivo_prueba[j] and i == 0):
        fn +=1
    j+=1
    
print("Buenos resultados:", vp+vn)
print("Malos resultados:", fp+fn)
print("La precision es: ", vp/(vp+fp))

30/30 [==============================] - 0s 1ms/step
Buenos resultados: 922
Malos resultados: 27
La precision es:  0.925531914893617


# El siguiente modelo a realizar es random forest (ChatGPT dice que es el mas sencillo)

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [14]:
# Cargar el archivo CSV
data = pandas.read_csv('Shill Bidding Dataset.csv')

# Separar los atributos (X) y el objetio (y)
# Ajusta 'etiqueta' al nombre de la columna objetivo
X = data[['Bidder_Tendency','Bidding_Ratio', 'Successive_Outbidding', 'Last_Bidding','Auction_Bids', 'Starting_Price_Average', 'Early_Bidding','Winning_Ratio', 'Auction_Duration']]
X = X.to_numpy()
y = data['Class']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# n_stimators = numero de arboles y max_depth profundidad maxima de cada arbol 
randomForest_Bidding = RandomForestClassifier(n_estimators=100, max_depth=10)
print(X)

[[2.00000000e-01 4.00000000e-01 0.00000000e+00 ... 2.77778000e-05
  6.66666667e-01 5.00000000e+00]
 [2.43902440e-02 2.00000000e-01 0.00000000e+00 ... 1.31226852e-02
  9.44444444e-01 5.00000000e+00]
 [1.42857143e-01 2.00000000e-01 0.00000000e+00 ... 3.04166670e-03
  1.00000000e+00 5.00000000e+00]
 ...
 [5.55555560e-02 4.34782610e-02 0.00000000e+00 ... 1.56630291e-02
  0.00000000e+00 7.00000000e+00]
 [7.69230770e-02 8.69565220e-02 0.00000000e+00 ... 4.15013200e-04
  0.00000000e+00 7.00000000e+00]
 [1.63934430e-02 4.34782610e-02 0.00000000e+00 ... 3.40350529e-01
  0.00000000e+00 7.00000000e+00]]


In [15]:
# Entrenamos el modelo con los datos
randomForest_Bidding.fit(X_train, y_train )

RandomForestClassifier(max_depth=10)

In [16]:
predictions = randomForest_Bidding.predict(X_test)
# Esto lo realizamos para poder visualizar una aproximacion
j = 0
vp=0
vn=0
fp=0
fn=0
for prediction, true_label in zip(predictions, y_test):
    if(prediction == true_label and prediction == 1  ):
        vp+=1
    elif(prediction != true_label and prediction == 1):
        fp +=1
    elif(prediction == true_label and prediction == 0):
        vn +=1
    elif(prediction != true_label and prediction == 0):
        fn +=1
    j+=1
    
print("Buenos resultados:", vp+vn)
print("Malos resultados:", fp+fn)
print("La precision es: ", vp/(vp+fp))


Buenos resultados: 1255
Malos resultados: 10
La precision es:  0.9357142857142857


# Este es el modelo LIME de interpretabilidad de modelos de caja negra

Este es el modelo de Lime que funciona podemos modificarlo averiguando la distancia con otro calculo como el de arriba y haciendo llamdas recursivas a funciones auxiliares

In [17]:
#Definitivo


#PARECE QUE ESTE TIENE BUENA PINTA 
def perturb(datos, cabecera):
    k = random.randint(1, len(datos))
    datos_perturbados = datos.copy()
    for j in range(k):
        atributo_a_perturbar = random.randint(0,len(datos)-1)
        rango_maximo = abs(max(a[atributo_a_perturbar] for a in cabecera))
        rango_minimo = abs(min(a[atributo_a_perturbar] for a in cabecera))
        datos_perturbados[atributo_a_perturbar] = random.uniform(rango_minimo, rango_maximo)
    return datos_perturbados

def explicacion_del_modelo(N,f,datos, cabecera):
    X = [] # Muestras perturbadas
    R = [] # Representaciones
    W = [] # Distancias entre muestra x y sus perturbaciones
    for i in range(N):
        datos_perturbados= perturb(datos, cabecera)
        w = abs(sum(datos[atributo] - datos_perturbados[atributo] for atributo in range(0, len(datos)-1)))
        r = [0 if datos_perturbados[atributo] == datos[atributo] else 1 for atributo in range(0,len(datos)-1)]
        X.append(datos_perturbados)
        R.append(r)
        W.append(w)
    X = np.expand_dims(datos_perturbados, axis=0)
    Y_perturbada = f.predict(X)
    G = Ridge()
    G.fit(R, Y_perturbada, sample_weight=W)
    return G.get_params()


In [19]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Ridge
# Ejemplo utilizando el modelo "red_bidding" definido en el código anterior
parametros = explicacion_del_modelo(1, red_bidding, atributos[5809], atributos)
params = explicacion_del_modelo(1, randomForest_Bidding, atributos[5809], atributos)
print(parametros)
print(params)

1/1 [==============================] - 0s 22ms/step
{'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'positive': False, 'random_state': None, 'solver': 'auto', 'tol': 0.0001}
{'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'positive': False, 'random_state': None, 'solver': 'auto', 'tol': 0.0001}


<p><strong>Alpha:</strong> Este parámetro controla la fuerza de la regularización en el modelo Ridge. Cuanto mayor sea el valor de alpha, mayor será la penalización aplicada a los coeficientes. Un valor de alpha igual a 1.0 indica una regularización moderada.</p>

<p><strong>Copy_X:</strong> Es un parámetro booleano que indica si los datos de entrada deben ser copiados antes de ajustar el modelo. Si se establece en True, se realiza una copia de los datos; si se establece en False, los datos se sobrescriben.</p>

<p><strong>Fit_intercept:</strong> Es un parámetro booleano que indica si se debe ajustar un intercepto en el modelo. Si se establece en True, el modelo incluirá un término constante; si se establece en False, el modelo no incluirá un término constante.</p>

<p><strong>Max_iter:</strong> Es el número máximo de iteraciones permitidas durante el ajuste del modelo. Si se establece en None, el modelo ajustará hasta que la convergencia sea alcanzada.</p>

<p><strong>Positive:</strong> Es un parámetro booleano que indica si los coeficientes deben ser no negativos. Si se establece en True, se restringe que los coeficientes sean siempre positivos.</p>

<p><strong>Random_state:</strong> Este parámetro controla la semilla utilizada por el generador de números aleatorios. Si se establece en None, el generador de números aleatorios utilizará una semilla diferente en cada ejecución.</p>

<p><strong>Solver:</strong> Especifica el algoritmo a utilizar en el ajuste del modelo. Si se establece en 'auto', el modelo seleccionará automáticamente el mejor algoritmo según los datos y el problema en cuestión.</p>

<p><strong>Tol:</strong> Especifica la tolerancia para el criterio de parada en el ajuste del modelo. Si la diferencia entre dos iteraciones consecutivas es menor que la tolerancia, el ajuste se considera convergido.</p>

# Aqui tenemos los calculos requeridos sobre el trabajo

 <h1>Resumen de cálculos</h1>

  <i><b>1. Identidad:</b></i>
  <p>Verifica si las explicaciones son idénticas para objetos idénticos.</p>

 <i><b>2. Separabilidad:</b></i>
  <p>Verifica que las explicaciones sean diferentes para objetos no idénticos.</p>

 <i><b>3. Estabilidad:</b></i>
  <p>Evalúa si objetos similares tienen explicaciones similares mediante la correlación de Spearman.</p>

  <i><b>4. Selectividad:</b></i>
  <p>Mide el efecto de eliminar variables relevantes en la predicción mediante el cálculo del área bajo la curva (AUC).</p>

  <i><b>5. Completitud (Coherencia):</b></i>
  <p>Calcula la coherencia de una muestra al eliminar características no importantes.</p>

  <i><b>6. Completitud (Coherencia / Explicación):</b></i>
  <p>Calcula la coherencia promedio y la desviación estándar de la coherencia.</p>

 <i><b>7. Completitud (Congruencia):</b></i>
  <p>Calcula la desviación estándar de la coherencia como un proxy de congruencia.</p>

<b> CABE DESTACAR QUE ESTOS CALCULOS ESTAN REALIZADOS DE MOMENTO CON VALORES ARBITRARIOS ES DECIR DEBEMOS DE OBTENER PRIMERO LOS VALORES DE LOS CALCULOS QUE DEBEMOS REALIZAR</b>

In [20]:

## Segunda vuelta de campana

# Distancias entre objetos
def object_distance(X_train, X_test, model):
    # Obtener las predicciones del modelo para los datos de entrenamiento y prueba
    predictions_train = model.predict(X_train)
    predictions_test = model.predict(X_test)
    predictions_train = predictions_train.reshape(-1, 1)
    predictions_test = predictions_test.reshape(-1, 1)
    # Calcular la distancia de objetos entre cada par de predicciones
    object_distances = cdist(predictions_train, predictions_test)
    
    return object_distances

# Distancias entre explicaciones
def explanation_distance(X_train, X_test, model):
    # Obtener las predicciones del modelo para los datos de entrenamiento y prueba
        
    predictions_train = randomForest_Bidding.predict(X_train)
    predictions_test = randomForest_Bidding.predict(X_test)

    # Calcular la distancia de explicaciones entre cada par de predicciones
    explanation_distances = np.linalg.norm(predictions_train - predictions_test, axis=1)

    return explanation_distances
        



def identidad_(dist_objetos, dist_explicaciones):
    identidad = np.all(dist_objetos == 0) and np.all(dist_explicaciones == 0)
    return identidad
    
def separabilidad_(dist_objetos, dist_explicaciones):
    separabilidad = np.all(dist_objetos != 0) and np.all(dist_explicaciones > 0)
    return separabilidad

def estabilidad_(dist_objetos, dist_explicaciones):
    correlacion = [spearmanr(dist_objetos[i], dist_explicaciones[i]).correlation for i in range(len(dist_objetos))]
    estabilidad = all(corr > 0 for corr in correlacion)
    return estabilidad

def selectividad_(muestras, n_caracteristicas, modelo):
    for i in range(n_caracteristicas):
        muestras_modificadas = np.copy(muestras)
        muestras_modificadas[:, i] = 0
        
        # Obtener las etiquetas binarias para la característica i
        y_true = np.where(muestras[:, i] == 0, 0, 1)
        y_true_binary = label_binarize(y_true, classes=np.unique(y_true))
        
        # Obtener las predicciones del modelo con la característica i eliminada
        y_score = modelo.predict(muestras_modificadas)
        
        # Calcular el área bajo la curva (AUC) y almacenarlo en auc_scores
        if len(np.unique(y_true_binary)) > 1:
            auc_scores.append(roc_auc_score(y_true_binary, y_score))
    
    if auc_scores:
        selectividad = np.mean(auc_scores)
    else:
        selectividad = 0.0
    
    return selectividad

def coherencia_(prediccion_original, prediccion_nueva):
    # Obtener las predicciones del modelo para la señal original y la nueva señal
    # Calcular el error de predicción para la señal original
    error_original = np.abs(predicciones_original - prediccion_nueva)
    # Calcular el error de predicción para la nueva señal
    error_nueva = np.abs(predicciones_nueva - y_nueva)
    coherencia = np.abs(prediction_original - prediction_nueva)
    return coherencia
def completitud_(prediccion_original, y_test):
    pi_e= np.mean(prediccion_original != y_test)
    ei_e = np.mean(np.abs(prediccion_original - y_original))
    completitud = np.abs(ei_e / pi_e) 
    return completitud
    
def congruencia_(coherencia):
    #Congruencia
    alpha = np.mean(completeness)  # alpha_values es una lista/array con los valores de coherencia alpha_i

    # Calcular la desviación estándar de la coherencia
    N = len(data)
    delta = np.sqrt(np.mean((completeness - alpha)**2) / N)  # N es el número total de muestras
    return delta

In [21]:
#Valores de funciones
predicciones_original = randomForest_Bidding.predict(X_train)
predicciones_nueva = randomForest_Bidding.predict(X_test)
dist_objetos = object_distance(X_train, X_test, randomForest_Bidding)
dist_explicaciones = explanation_distance(X_train, X_test, randomForest_Bidding)

ValueError: operands could not be broadcast together with shapes (5056,) (1265,) 

In [22]:
object_distance(X_train,X_test, randomForest_Bidding)

array([[1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])